In [ ]:
import sys
from pathlib import Path
from os.path import dirname, abspath
d = dirname(abspath(Path.cwd()))
sys.path.insert(0, d)
import hyper_params
import pickle
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt
from Utils.delong_auc_test import delong_roc_test
import numpy as np
from rpy2 import robjects
import rpy2.robjects.packages as rpackages

utils = rpackages.importr('utils')
utils.install_packages("pROC")

with open(hyper_params.LOCATION_OF_RESULTS_ON_TEST_GROUP, 'rb') as f:
    results = pickle.load(f)


In [ ]:
relevant_persons = set(results['rnn']['Person indices'])
for model_name in results:
    relevant_persons = set(results[model_name]['Person indices']).intersection(relevant_persons)
get_model_info = lambda model: {p: (prediction, ground_truth) for p, prediction, ground_truth in 
                                     zip(results[model]['Person indices'], results[model]['Estimated score'], results[model]['Ground truth'])}

In [ ]:
ax = plt.gca()
new_names = {'rnn': "RNN", "lstm": "LSTM", "XGBoost": "XGBoost", "bsi_mimiciv_train": "RatchetEHR"}
for model_name in results:
    if model_name in new_names:
        model_info = get_model_info(model_name)
        model_info = [model_info[p] for p in sorted(relevant_persons, key= lambda x: x[1])]
        RocCurveDisplay.from_predictions(y_true = [x[1] for x in model_info], 
                                        y_pred = [x[0] for x in model_info], 
                                        name = new_names[model_name],ax = ax)

In [ ]:
import sklearn
import scipy
%load_ext rpy2.ipython
for first_model_name in results:
    for second_model_name in results:
        if first_model_name == 'bsi_mimiciv_train' and first_model_name != second_model_name and first_model_name in new_names and second_model_name in new_names:
            first_model_info = get_model_info(first_model_name)
            first_model_info = [first_model_info[p] for p in sorted(relevant_persons, key= lambda x: x[1])]
            second_model_info = get_model_info(second_model_name)
            second_model_info = [second_model_info[p] for p in sorted(relevant_persons, key= lambda x: x[1])]
            assert([x[1] for x in first_model_info] == [x[1] for x in second_model_info])
            print(len([x[1] for x in first_model_info if x[1] == 1]) / len([x[1] for x in first_model_info]))
            print("p-value for ", new_names[first_model_name], 
                  " VS ", new_names[second_model_name])
            r_text = '''library(pROC)''' +'\n'+ \
            '''response<-c''' + str(tuple([x[1] for x in first_model_info])) +'\n' +\
                '''modela<-c''' + str(tuple([x[0] for x in first_model_info])) + '\n' +\
                '''modelb<-c''' + str(tuple([x[0] for x in second_model_info])) + '\n' + \
                '''roca <- roc(response,modela)''' + '\n' + \
                '''rocb<-roc(response,modelb)''' + '\n' + \
                '''roc.test(roca,rocb,method=c("delong"), boot.n = 1500)'''
            get_ipython().run_cell_magic(
                "R",
                "",
                r_text
                )

In [ ]:

# robjects.r('''
#     library(pROC)
#     add_nums = roc([0.5], [1])
    
#     print(add_nums)
# ''')